<a href="https://colab.research.google.com/github/manish190502/financial-fraud-detection-using-generative-AI/blob/main/cgan_wiht_4_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense, Concatenate, LSTM, Conv1D, MaxPooling1D, Flatten, GRU
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load your dataset
data = pd.read_excel('/content/drive/MyDrive/fraudset.xlsx')

# Preprocessing
X = data.drop('Class', axis=1).values  # Features
y = data['Class'].values.reshape(-1, 1)  # Target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)











In [13]:
# Generator model
gen_input_dim = X_train_scaled.shape[1]
gen_input = Input(shape=(gen_input_dim,))
gen = Dense(128, activation='relu')(gen_input)
gen = Dense(64, activation='relu')(gen)
gen_output = Dense(gen_input_dim, activation='tanh')(gen)
generator = Model(gen_input, gen_output)

# Discriminator model
disc_input_dim = gen_input_dim + 1  # Features + Class label
disc_input = Input(shape=(disc_input_dim,))
disc = Dense(128, activation='relu')(disc_input)
disc = Dense(64, activation='relu')(disc)
disc_output = Dense(1, activation='sigmoid')(disc)
discriminator = Model(disc_input, disc_output)
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

# Combined model (CGAN)
cgan_input = Input(shape=(gen_input_dim,))
label_input = Input(shape=(1,))
concatenated_input = Concatenate()([cgan_input, label_input])
cgan_output = discriminator(concatenated_input)
cgan = Model(inputs=[cgan_input, label_input], outputs=cgan_output)
cgan.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train CGAN
batch_size = 128
epochs = 10000
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, X_train_scaled.shape[0], batch_size)
    real_transactions = X_train_scaled[idx]
    real_labels = y_train[idx]
    fake_labels = np.random.randint(0, 2, (batch_size, 1))
    fake_transactions = generator.predict(np.random.normal(0, 1, (batch_size, gen_input_dim)))
    disc_loss_real = discriminator.train_on_batch(np.concatenate([real_transactions, real_labels], axis=1), np.ones((batch_size, 1)))
    disc_loss_fake = discriminator.train_on_batch(np.concatenate([fake_transactions, fake_labels], axis=1), np.zeros((batch_size, 1)))
    disc_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, gen_input_dim))
    valid_y = np.ones((batch_size, 1))
    gen_loss = cgan.train_on_batch([noise, fake_labels], valid_y)

    # Print progress
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Disc_loss: {disc_loss[0]}, Gen_loss: {gen_loss}')

# Generate synthetic data
num_synthetic_samples = 1000
synthetic_noise = np.random.normal(0, 1, (num_synthetic_samples, gen_input_dim))
synthetic_labels = np.random.randint(0, 2, (num_synthetic_samples, 1))
synthetic_data = generator.predict(synthetic_noise)

# Save synthetic data to a CSV file
synthetic_df = pd.DataFrame(synthetic_data, columns=data.columns[:-1])  # Assuming last column is the target 'Class'
synthetic_df.to_csv('fraudgan.csv', index=False)

# Load synthetic data
synthetic_data = pd.read_csv('fraudgan.csv')


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 3ms/step
Epoch: 5100, Disc_loss: 1.0310872183544979e-06, Gen_loss: [2.3624261184448936e-13, 1.0]
4/4 [==============================] - 0s 4ms/step
Epoch: 5200, Disc_loss: 3.554243053408651e-06, Gen_loss: [6.159245856396378e-12, 1.0]
4/4 [==============================] - 0s 3ms/step
Epoch: 5300, Disc_loss: 5.314579084279508e-07, Gen_loss: [8.332082579443068e-09, 1.0]
4/4 [==============================] - 0s 3ms/step
Epoch: 5400, Disc_loss: 1.7191071712633612e-06, Gen_loss: [7.803684367505123e-16, 1.0]
4/4 [==============================] - 0s 3ms/step
Epoch: 5500, Disc_loss: 1.60307227091422e-06, Gen_loss: [5.860556441964725e-18, 1.0]
4/4 [==============================] - 0s 3ms/step
Epoch: 5600, Disc_loss: 3.608454846926179e-06, Gen_loss: [8.45010571389436e-14, 1.0]
4/4 [==============================] - 0s 3ms/step
Epoch: 5700, Disc_loss: 7.74860644121118e-06, Gen_loss: [1.0095180186908692

In [15]:
# Preprocessing
scaler = StandardScaler()
scaled_data = scaler.fit_transform(synthetic_data)

# Define sequence length
sequence_length = 10

# Generate sequences
sequences = []
for i in range(len(scaled_data) - sequence_length):
    sequences.append(scaled_data[i:i + sequence_length])

# Convert to numpy array
sequences = np.array(sequences)

In [16]:
# Split sequences into features and target
X_rnn = sequences[:, :-1]    # Features
y_true_rnn = sequences[:, -1]   # Target (next data point in the sequence)

# Define and train RNN model
rnn_model = Sequential()
rnn_model.add(LSTM(units=64, input_shape=(X_rnn.shape[1], X_rnn.shape[2])))
rnn_model.add(Dense(units=X_rnn.shape[2]))  # Output layer
rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit(X_rnn, y_true_rnn, epochs=10, batch_size=32)

# Make predictions for the next data point in the sequence
rnn_predicted_next_point = rnn_model.predict(X_rnn)

# Calculate reconstruction errors for RNN
rnn_reconstruction_errors = np.mean(np.abs(rnn_predicted_next_point - y_true_rnn), axis=1)

# Set anomaly threshold for RNN
rnn_threshold = np.mean(rnn_reconstruction_errors) + 3 * np.std(rnn_reconstruction_errors)

# Detect anomalies using RNN
rnn_anomalies = np.where(rnn_reconstruction_errors > rnn_threshold)[0]

# Print anomalies detected by RNN
print("Anomalies indices (RNN):", rnn_anomalies)

# Define true labels for anomalies
true_anomalies = np.zeros_like(y_true_rnn)
true_anomalies[rnn_anomalies] = 1

# Define predicted labels for RNN anomalies
predicted_rnn_anomalies = np.zeros_like(y_true_rnn)
predicted_rnn_anomalies[rnn_anomalies] = 1

Epoch 1/10
31/31 [==============================] - 2s 6ms/step - loss: 1.0198
Epoch 2/10
31/31 [==============================] - 0s 6ms/step - loss: 1.0002
Epoch 3/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9896
Epoch 4/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9808
Epoch 5/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9722
Epoch 6/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9631
Epoch 7/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9539
Epoch 8/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9437
Epoch 9/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9325
Epoch 10/10
31/31 [==============================] - 0s 3ms/step
Anomalies indices (RNN): [115 175 213 353 847]


In [17]:
# Define and train CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_rnn.shape[1], X_rnn.shape[2])))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=X_rnn.shape[2]))
cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.fit(X_rnn, y_true_rnn, epochs=10, batch_size=32)

# Make predictions for the next data point in the sequence
cnn_predicted_next_point = cnn_model.predict(X_rnn)

# Calculate reconstruction errors for CNN
cnn_reconstruction_errors = np.mean(np.abs(cnn_predicted_next_point - y_true_rnn), axis=1)

# Set anomaly threshold for CNN
cnn_threshold = np.mean(cnn_reconstruction_errors) + 3 * np.std(cnn_reconstruction_errors)

# Detect anomalies using CNN
cnn_anomalies = np.where(cnn_reconstruction_errors > cnn_threshold)[0]

# Print anomalies detected by CNN
print("Anomalies indices (CNN):", cnn_anomalies)

# Define true labels for anomalies
true_anomalies = np.zeros_like(y_true_rnn)
true_anomalies[cnn_anomalies] = 1

# Define predicted labels for CNN anomalies
predicted_cnn_anomalies = np.zeros_like(y_true_rnn)
predicted_cnn_anomalies[cnn_anomalies] = 1


Epoch 1/10
31/31 [==============================] - 1s 2ms/step - loss: 1.4670
Epoch 2/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1997
Epoch 3/10
31/31 [==============================] - 0s 2ms/step - loss: 1.1071
Epoch 4/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0487
Epoch 5/10
31/31 [==============================] - 0s 2ms/step - loss: 1.0081
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9790
Epoch 7/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9546
Epoch 8/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9337
Epoch 9/10
31/31 [==============================] - 0s 2ms/step - loss: 0.9144
Epoch 10/10
31/31 [==============================] - 0s 1ms/step
Anomalies indices (CNN): [213 368 369 414 861 902]


In [18]:
# Define and train FCNN model
fcnn_model = Sequential()
fcnn_model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))  # Adjusted input shape
fcnn_model.add(Dense(64, activation='relu'))
fcnn_model.add(Dense(1, activation='sigmoid'))
fcnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
fcnn_model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

# Make predictions for FCNN
fcnn_predicted = fcnn_model.predict(X_test_scaled)
fcnn_predicted_binary = np.round(fcnn_predicted)

# Calculate evaluation metrics for FCNN
fcnn_accuracy = accuracy_score(y_test, fcnn_predicted_binary)
fcnn_precision = precision_score(y_test, fcnn_predicted_binary)
fcnn_recall = recall_score(y_test, fcnn_predicted_binary)
fcnn_f1 = f1_score(y_test, fcnn_predicted_binary)


Epoch 1/10
25/25 [==============================] - 1s 3ms/step - loss: 0.4357 - accuracy: 0.8209
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.1676 - accuracy: 0.9533
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0903 - accuracy: 0.9748
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0575 - accuracy: 0.9861
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0399 - accuracy: 0.9912
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0294 - accuracy: 0.9912
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0214 - accuracy: 0.9950
Epoch 8/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.9962
Epoch 9/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 0.9962
Epoch 10/10
7/7 [==============================] - 0s 2ms/step


In [19]:
# Define and train GRU model
gru_model = Sequential()
gru_model.add(GRU(units=64, input_shape=(X_rnn.shape[1], X_rnn.shape[2])))
gru_model.add(Dense(units=X_rnn.shape[2]))  # Output layer
gru_model.compile(optimizer='adam', loss='mse')
gru_model.fit(X_rnn, y_true_rnn, epochs=10, batch_size=32)

# Make predictions for the next data point in the sequence using GRU
gru_predicted_next_point = gru_model.predict(X_rnn)

# Calculate reconstruction errors for GRU
gru_reconstruction_errors = np.mean(np.abs(gru_predicted_next_point - y_true_rnn), axis=1)

# Set anomaly threshold for GRU
gru_threshold = np.mean(gru_reconstruction_errors) + 3 * np.std(gru_reconstruction_errors)

# Detect anomalies using GRU
gru_anomalies = np.where(gru_reconstruction_errors > gru_threshold)[0]

# Print anomalies detected by GRU
print("Anomalies indices (GRU):", gru_anomalies)



Epoch 1/10
31/31 [==============================] - 3s 10ms/step - loss: 1.0618
Epoch 2/10
31/31 [==============================] - 0s 9ms/step - loss: 1.0189
Epoch 3/10
31/31 [==============================] - 0s 10ms/step - loss: 1.0006
Epoch 4/10
31/31 [==============================] - 0s 10ms/step - loss: 0.9883
Epoch 5/10
31/31 [==============================] - 0s 9ms/step - loss: 0.9784
Epoch 6/10
31/31 [==============================] - 0s 7ms/step - loss: 0.9697
Epoch 7/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9612
Epoch 8/10
31/31 [==============================] - 0s 5ms/step - loss: 0.9536
Epoch 9/10
31/31 [==============================] - 0s 6ms/step - loss: 0.9452
Epoch 10/10
31/31 [==============================] - 0s 2ms/step
Anomalies indices (GRU): [213 346 369 626 847]


In [20]:
# Define true labels for anomalies
true_anomalies = np.zeros_like(y_true_rnn)
true_anomalies[gru_anomalies] = 1


# Define predicted labels for GRU anomalies
predicted_gru_anomalies = np.zeros_like(y_true_rnn)
predicted_gru_anomalies[gru_anomalies] = 1

rnn_accuracy = accuracy_score(true_anomalies, predicted_rnn_anomalies)
rnn_precision = precision_score(true_anomalies, predicted_rnn_anomalies, average=None)[1]
rnn_recall = recall_score(true_anomalies, predicted_rnn_anomalies, average=None)[1]
rnn_f1 = f1_score(true_anomalies, predicted_rnn_anomalies, average=None)[1]



# Compute evaluation metrics for CNN
cnn_accuracy = accuracy_score(true_anomalies, predicted_cnn_anomalies)
cnn_precision = precision_score(true_anomalies, predicted_cnn_anomalies,average=None)[1]
cnn_recall = recall_score(true_anomalies, predicted_cnn_anomalies,average=None)[1]
cnn_f1 = f1_score(true_anomalies, predicted_cnn_anomalies,average=None)[1]

# Compute evaluation metrics for FCNN
fcnn_accuracy = accuracy_score(y_test, fcnn_predicted_binary)
fcnn_precision = precision_score(y_test, fcnn_predicted_binary,average=None)[1]
fcnn_recall = recall_score(y_test, fcnn_predicted_binary,average=None)[1]
fcnn_f1 = f1_score(y_test, fcnn_predicted_binary,average=None)[1]

# Compute evaluation metrics for GRU
gru_accuracy = accuracy_score(true_anomalies, predicted_gru_anomalies)
gru_precision = precision_score(true_anomalies, predicted_gru_anomalies,average=None)[1]
gru_recall = recall_score(true_anomalies, predicted_gru_anomalies,average=None)[1]
gru_f1 = f1_score(true_anomalies, predicted_gru_anomalies,average=None)[1]

print("Evaluation Metrics for RNN:")
print("Accuracy:", rnn_accuracy)
print("Precision:", rnn_precision)
print("Recall:", rnn_recall)
print("F1 Score:", rnn_f1)

print("\nEvaluation Metrics for CNN:")
print("Accuracy:", cnn_accuracy)
print("Precision:", cnn_precision)
print("Recall:", cnn_recall)
print("F1 Score:", cnn_f1)

print("\nEvaluation Metrics for FCNN:")
print("Accuracy:", fcnn_accuracy)
print("Precision:", fcnn_precision)
print("Recall:", fcnn_recall)
print("F1 Score:", fcnn_f1)

print("\nEvaluation Metrics for GRU:")
print("Accuracy:", gru_accuracy)
print("Precision:", gru_precision)
print("Recall:", gru_recall)
print("F1 Score:", gru_f1)


Evaluation Metrics for RNN:
Accuracy: 0.9939393939393939
Precision: 0.4
Recall: 0.4
F1 Score: 0.4000000000000001

Evaluation Metrics for CNN:
Accuracy: 0.9929292929292929
Precision: 0.3333333333333333
Recall: 0.4
F1 Score: 0.3636363636363636

Evaluation Metrics for FCNN:
Accuracy: 0.9949748743718593
Precision: 0.9893617021276596
Recall: 1.0
F1 Score: 0.9946524064171123

Evaluation Metrics for GRU:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
